PREDICTIVE MODEL FOR DETERMINING THE PRICE OF A CAR

In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
#import train dataset
train=pd.read_csv('Train.csv')

In [3]:
#import test dataset which will be used for creating predicted values for submission
test=pd.read_csv('Test.csv')

In [4]:
#Read the first 5 rows
train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [5]:
#Take out the comma values in the Year column of the train dataset 
train['Year']=train['Year'].str.replace(',','')

In [6]:
#Take out the comma values in the Distance column of the train dataset
train['Distance']=train['Distance'].str.replace(',','')

In [7]:
#Ensure the commas are out
train.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,2011,Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,2012,Silver,3.5,Nigerian Used,125000
2,VHL19499,Lagos,Lexus,RX 350,2010,Red,9.2,Foreign Used,110852
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,2017,Blue,22.8,Foreign Used,30000
4,VHL12170,Ibadan,Toyota,Highlander,2002,Red,2.6,Nigerian Used,125206


In [8]:
#Get the info on the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7184 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                4845 non-null   object 
dtypes: float64(1), object(8)
memory usage: 506.7+ KB


In [9]:
#Observe that the year column and Distance aren't still numerical

In [10]:
#Make the train year column numerical
train['Year'] = train['Year'].apply (pd.to_numeric, errors='coerce')

In [11]:
#Make the test year column numerical
train['Distance'] = train['Distance'].apply (pd.to_numeric, errors='coerce')

In [12]:
#dropping NaN columns
train=train.dropna()

In [13]:
#info on the dataset with dropped column
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4627 entries, 1 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               4627 non-null   object 
 1   Location                4627 non-null   object 
 2   Maker                   4627 non-null   object 
 3   Model                   4627 non-null   object 
 4   Year                    4627 non-null   float64
 5   Colour                  4627 non-null   object 
 6   Amount (Million Naira)  4627 non-null   float64
 7   Type                    4627 non-null   object 
 8   Distance                4627 non-null   float64
dtypes: float64(3), object(6)
memory usage: 361.5+ KB


In [14]:
#checking how many unique values are in that column
train['Model'].nunique()

952

In [15]:
#checking how many unique values are in that column
train['Location'].nunique()

3

In [16]:
#checking how many unique values are in that column
train['Maker'].nunique()

49

In [17]:
#dropping the VehicleID Column
train=train.drop('VehicleID', axis=1)


In [18]:
#Using Label Encoder
train['Maker']=LabelEncoder().fit_transform(train['Maker'])

In [19]:
train.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
1,Ibadan,16,Sonata,2012.0,Silver,3.50,Nigerian Used,125000.0
2,Lagos,25,RX 350,2010.0,Red,9.20,Foreign Used,110852.0
3,Abuja,30,GLE-Class,2017.0,Blue,22.80,Foreign Used,30000.0
4,Ibadan,46,Highlander,2002.0,Red,2.60,Nigerian Used,125206.0
8,Lagos,46,Sienna,2012.0,Blue,7.76,Foreign Used,350882.0


In [20]:
#using label encoder to transform the Model Column and Maker Column
#Yes, Label encoder shouldn't be used for the inputs but this particular dataset has too many unique values in these columns
#...so we  don't have an option because using OneHotEncoder will create columns that won't match the number of columns
#... in the test dataset
train['Model']=LabelEncoder().fit_transform(train['Model'])

In [21]:
train.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
1,Ibadan,16,828,2012.0,Silver,3.50,Nigerian Used,125000.0
2,Lagos,25,718,2010.0,Red,9.20,Foreign Used,110852.0
3,Abuja,30,391,2017.0,Blue,22.80,Foreign Used,30000.0
4,Ibadan,46,440,2002.0,Red,2.60,Nigerian Used,125206.0
8,Lagos,46,801,2012.0,Blue,7.76,Foreign Used,350882.0


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4627 entries, 1 to 7204
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Location                4627 non-null   object 
 1   Maker                   4627 non-null   int32  
 2   Model                   4627 non-null   int32  
 3   Year                    4627 non-null   float64
 4   Colour                  4627 non-null   object 
 5   Amount (Million Naira)  4627 non-null   float64
 6   Type                    4627 non-null   object 
 7   Distance                4627 non-null   float64
dtypes: float64(3), int32(2), object(3)
memory usage: 289.2+ KB


In [23]:
#creating the feature dataset
X=train.drop('Amount (Million Naira)', axis=1)

In [24]:
#creating the output dataset
y=train['Amount (Million Naira)']

In [25]:
#Encoding the rest of the columns 
cat=['Location', 'Colour','Type']
transformer=ColumnTransformer([('one_hot', OneHotEncoder(), cat)], remainder='passthrough')
X=transformer.fit_transform(X)

In [26]:
#size of the new feature dataset
X.shape

(4627, 28)

In [27]:
#splitting the dataset into train and test
x_train,x_test, y_train, y_test=train_test_split(X, y, test_size=0.33, random_state=89)

In [28]:
#creating the train model
lr_model=LinearRegression()

In [29]:
lr_model.fit(x_train, y_train)

LinearRegression()

In [30]:
#testing the accuracy of the model
y_pred=lr_model.predict(x_test)

In [31]:
#viewing the first five rows
y_pred[:5]

array([ 4.43142557, 15.37766471, 14.55779517, 14.71160916,  5.08927927])

In [32]:
#accuracy of the model...r2 score, very poor but its the first try
#subsequently, we'll take off some columns that i think may be causing the low r2 score. 
#E.g the Maker and Model columns are problematic... 
#Something else that can be done is using the mean of the Distance column at the begining. Remember, I dropped all
#...those in this work. Generally we'll find  way to fill the dataset instead of deleting them. 
accuracy=r2_score(y_test, y_pred)
print(accuracy)

0.33388385786603203


# TEST DATASET PREPROCESSING

FOR THE TEST DATASET,  I CLEANED IT UP AND ENSURED THAT THE COLUMNS AFTER CLEANING MATCHES THE ONES FOR THE TRAIN DATASET. THIS IS VERY IMPORTANT ELSE I WON'T BE ABLE TO APPLY THE  MODEL ON THE DATASET. INFACT IT IS BECAUSE OF THIS THAT I WENT BACK TO USE LABEL ENCODER IN SOME PARTS BECAUSE I GOT VARRYING COLUMN SIZES AT THE END AFTER USING ONE HOT ENCODER BUT I REALSISED THAT THOSE COLUMNS WITH TOO MANY VARRYING VALUES COULD BE THE PROBLEM BECAUSE SOME VALUES CONTAINED IN THE TRAIN VALUES MAY NOT BE THE SAME AS THE SAME IN THE TEST DATASET. 

In [33]:
test.head()

,VehicleID,Location,Maker,Model,Year,Colour,Type,Distance
0,VHL18518,Abuja,BMW,323i,"2,008",White,Foreign Used,30524.0
1,VHL17149,Lagos,Toyota,Camry,"2,013",White,Foreign Used,NaN
2,VHL10927,Lagos,Toyota,Highlander Limited V6,"2,005",Gold,Foreign Used,NaN
3,VHL12909,Lagos,Toyota,Camry,"2,011",Gray,Foreign Used,166839.0
4,VHL12348,Lagos,Lexus,ES 350 FWD,"2,013",Red,Foreign Used,88862.0


In [34]:
#taking out the comma from the Year column
test['Year']=test['Year'].str.replace(',','')

In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   VehicleID  2061 non-null   object 
 1   Location   2061 non-null   object 
 2   Maker      2061 non-null   object 
 3   Model      2061 non-null   object 
 4   Year       2059 non-null   object 
 5   Colour     2061 non-null   object 
 6   Type       2007 non-null   object 
 7   Distance   1385 non-null   float64
dtypes: float64(1), object(7)
memory usage: 128.9+ KB


In [36]:
#making the year column float too
test['Year'] = test['Year'].apply (pd.to_numeric, errors='coerce')

In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   VehicleID  2061 non-null   object 
 1   Location   2061 non-null   object 
 2   Maker      2061 non-null   object 
 3   Model      2061 non-null   object 
 4   Year       2059 non-null   float64
 5   Colour     2061 non-null   object 
 6   Type       2007 non-null   object 
 7   Distance   1385 non-null   float64
dtypes: float64(2), object(6)
memory usage: 128.9+ KB


In [38]:
#Encoding the Maker and Model Columns
test['Maker']=LabelEncoder().fit_transform(test['Maker'])

In [39]:
test['Model']=LabelEncoder().fit_transform(test['Model'])

In [40]:
#Checking the missing values
test.isna().sum()

VehicleID      0
Location       0
Maker          0
Model          0
Year           2
Colour         0
Type          54
Distance     676
dtype: int64

In [41]:
#In the quest to fill in missing values with their mode and mean, we do the following

In [42]:
mode_y=test.Year.mode()[0]
print(mode_y)

2008.0


In [43]:
mode_t=test.Type.mode()[0]
print(mode_t)

Foreign Used


In [44]:
mean_d=test.Distance.mean()
print(mean_d)

103800.66859205776


In [45]:
#Filling the Year column with the mode
test['Year'].fillna(mode_y, inplace=True)

In [46]:
#Filling the missing values in the Type column with the mode
test['Type'].fillna(mode_t, inplace=True)

In [47]:
#Filling the missing values in the Distance column with the mean
test['Distance'].fillna(mean_d, inplace=True)

In [48]:
test.isna().sum()

VehicleID    0
Location     0
Maker        0
Model        0
Year         0
Colour       0
Type         0
Distance     0
dtype: int64

In [49]:
#Dropping the ID Column
test.drop('VehicleID', axis=1, inplace=True)

In [50]:
test.head()

,Location,Maker,Model,Year,Colour,Type,Distance
0,Abuja,2,8,2008.0,White,Foreign Used,30524.000000
1,Lagos,37,123,2013.0,White,Foreign Used,103800.668592
2,Lagos,37,272,2005.0,Gold,Foreign Used,103800.668592
3,Lagos,37,123,2011.0,Gray,Foreign Used,166839.000000
4,Lagos,20,192,2013.0,Red,Foreign Used,88862.000000


In [51]:
#Encoding these columns with few unique values
cat_test=['Location', 'Colour','Type']
transformer_test=ColumnTransformer([('one_hot_test', OneHotEncoder(), cat_test)], remainder='passthrough')
test_trans=transformer.fit_transform(test)

In [52]:
#Compaaring the test data column and the one used for cheking the performance of the model
#this test trans is the TEST Dtatset, the one we are to put the input to get an output to submit.
test_trans.shape

(2061, 28)

In [53]:
x_test.shape

(1527, 28)

In [54]:
test_submission=lr_model.predict(test_trans)

In [55]:
test_submission

array([ 6.30920963, 11.20655073, -1.59066698, ..., 14.03705094,
       14.32786052,  8.12940948])

In [56]:
submission=pd.read_csv("SampleSubmission.csv")
submission['Amount (Million Naira)']=test_submission

In [57]:
submission.to_csv('first_submission.csv', index=False)